## Install Packages

In [1]:
!conda install -y -c conda-forge faiss-gpu
!apt-get -y update
!apt-get -y install libatlas-base-dev

done
Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - faiss-gpu


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.14.0               |   py37h89c1867_0        1010 KB  conda-forge
    toolz-0.12.1               |     pyhd8ed1ab_0          51 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         1.0 MB

The following NEW packages will be INSTALLED:

  toolz              conda-forge/noarch::toolz-0.12.1-pyhd8ed1ab_0

The following packages will be UPDATED:

  conda                               4.12.0-py37h89c1867_0 --> 4.14.0-py37h89c1867_0



toolz-0.12.1         | 51 KB     | ##################################### | 100% 
conda-4.14.0         | 1010 KB   | ##################################### | 100% 
Preparing transaction: done
Ver

## Load Data

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.random_projection import GaussianRandomProjection

from tqdm import tqdm

import faiss

In [3]:
df = pd.read_csv("story_dataset.csv")
df

,prompt_id,prompt,story,hidden_state_file,len_generated_story,len_new_story
0,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Blaz...,./hidden_states/prompt_1.npz,270,271
1,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Spar...,./hidden_states/prompt_1.npz,349,350
2,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Scor...,./hidden_states/prompt_1.npz,278,278
3,1,Once upon a time there was a dragon,Once upon a time there was a dragon. The drago...,./hidden_states/prompt_1.npz,117,118
4,1,Once upon a time there was a dragon,Once upon a time there was a dragon. The drago...,./hidden_states/prompt_1.npz,129,130
...,...,...,...,...,...,...
9995,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,289,290
9996,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,119,119
9997,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,127,128
9998,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,441,441


In [4]:
max_story_len = max(df["len_generated_story"])
max_story_len

522

In [5]:
hidden_states_by_layer = {}
NUM_PROMPTS = 10

for prompt_id in range(1, NUM_PROMPTS + 1):
    with np.load(f'./hidden_states/prompt_{prompt_id}.npz') as loaded_data:
        for i in tqdm(range(1000)):
            curr_hidden_states = loaded_data[f"arr_{i}"][0]
#             print(curr_hidden_states.shape)
            for layer in range(6, 7):
                padded_arr = np.zeros((max_story_len, 512))
                padded_arr_len = len(curr_hidden_states[layer][0])
                
                padded_arr[:padded_arr_len] = curr_hidden_states[layer][0]
                
                padded_arr = padded_arr.flatten().astype('float32') #FAISS expects data in type float32 instead of float64 - saves memory too!
#                 print(padded_arr.shape)
                
                if(f"layer_{layer}" in hidden_states_by_layer):
                    hidden_states_by_layer[f"layer_{layer}"].append(padded_arr)
                else:
                    hidden_states_by_layer[f"layer_{layer}"] = [padded_arr]

100%|███████████████████████████████████████████████████████████████████████████████| 1000/1000 [04:17<00:00,  3.89it/s]


In [6]:
layer_hs_array = np.array(hidden_states_by_layer["layer_6"])
layer_hs_array.shape

(10000, 267264)

## Layer 6 Clustering

In [7]:
# Use original vectors for clustering - uncomment next line and comment out last two lines

dim_reduced_vecs = layer_hs_array

# random_projector = GaussianRandomProjection(random_state = 42)
# dim_reduced_vecs = random_projector.fit_transform(layer_hs_array).astype('float32')

In [8]:
dim_reduced_vecs = np.array([v / np.linalg.norm(v) for v in dim_reduced_vecs])
dim_reduced_vecs.shape

(10000, 267264)

In [9]:
# K-means Clustering

ncentroids = NUM_PROMPTS
niter = 20
verbose = True
dim = dim_reduced_vecs.shape[1]
kmeans = faiss.Kmeans(dim, ncentroids, niter = niter, verbose = verbose, gpu = True, nredo = 10, spherical = True, max_points_per_centroid = 1000)
kmeans.train(dim_reduced_vecs)

Clustering 10000 points in 267264D to 10 clusters, redo 10 times, 20 iterations
  Preprocessing in 1.66 s
Outer iteration 0 / 10
  Iteration 19 (27.68 s, search 18.62 s): objective=5193.91 imbalance=1.455 nsplit=0       
Objective improved: keep new clusters
Outer iteration 1 / 10
  Iteration 19 (55.25 s, search 37.24 s): objective=5204.81 imbalance=1.583 nsplit=0       
Objective improved: keep new clusters
Outer iteration 2 / 10
  Iteration 19 (82.93 s, search 55.86 s): objective=5197.17 imbalance=1.211 nsplit=0       
Outer iteration 3 / 10
  Iteration 19 (110.58 s, search 74.48 s): objective=5194.5 imbalance=1.208 nsplit=0        
Outer iteration 4 / 10
  Iteration 19 (138.46 s, search 93.09 s): objective=5172.19 imbalance=1.158 nsplit=0       
Outer iteration 5 / 10
  Iteration 19 (166.25 s, search 111.68 s): objective=5191.1 imbalance=1.496 nsplit=0        
Outer iteration 6 / 10
  Iteration 19 (193.99 s, search 130.34 s): objective=5177.29 imbalance=1.465 nsplit=0       
Outer i

5204.8134765625

In [10]:
kmeans.centroids #cluster centers

array([[-2.1314079e-02,  2.3812255e-02,  9.6693868e-03, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [-1.7484300e-02,  1.9533621e-02,  7.9319710e-03, ...,
         8.7589370e-07,  1.6370795e-06, -1.4851837e-06],
       [-2.0963062e-02,  2.3420122e-02,  9.5101455e-03, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       ...,
       [-2.1144137e-02,  2.3622394e-02,  9.5922872e-03, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [-2.1208586e-02,  2.3694402e-02,  9.6215354e-03, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [-1.7506249e-02,  1.9558126e-02,  7.9419222e-03, ...,
        -2.5244344e-05,  2.3833207e-05, -1.3124142e-05]], dtype=float32)

In [11]:
for centroid in kmeans.centroids:
    print(np.linalg.norm(centroid))

1.000008
1.0000135
1.0000064
1.0000284
1.0000063
1.0000079
1.0000205
1.0000055
1.0000093
1.0000198


In [12]:
kmeans.obj #inertia at each iteration

array([3056.67016602, 5009.83544922, 5126.23974609, 5168.97265625,
       5180.92578125, 5185.17382812, 5188.20703125, 5190.59667969,
       5191.42578125, 5191.90722656, 5192.24511719, 5192.43554688,
       5192.59472656, 5192.77832031, 5192.95361328, 5193.11474609,
       5193.31103516, 5193.46533203, 5193.63671875, 5193.91015625,
       3175.83056641, 4967.78759766, 5108.10205078, 5179.06542969,
       5191.20654297, 5196.27441406, 5198.47363281, 5199.61035156,
       5201.04248047, 5202.08007812, 5202.45556641, 5202.66894531,
       5202.91992188, 5203.54980469, 5204.40136719, 5204.68408203,
       5204.73486328, 5204.76220703, 5204.78955078, 5204.81347656])

In [13]:
normalized_vecs = [v / np.linalg.norm(v) for v in dim_reduced_vecs]

In [14]:
cos_similarities = normalized_vecs @ kmeans.centroids.T
classifications = np.argmax(cos_similarities, axis=1)

In [15]:
pd.Series(classifications).value_counts()

1    2675
4    1907
6    1724
8     774
5     565
0     554
9     546
3     434
7     412
2     409
dtype: int64

In [16]:
pd.Series(kmeans.index.search(dim_reduced_vecs.astype(np.float32), 1)[1].flatten()).value_counts()

1    2675
4    1907
6    1724
8     774
5     565
0     554
9     546
3     434
7     412
2     409
dtype: int64

In [17]:
prompt_ids = df["prompt_id"]
prompt_ids = prompt_ids.to_numpy()
prompt_ids

array([ 1,  1,  1, ..., 10, 10, 10])

In [18]:
# Get most common centroid for each 1000 points (same label)
max_centroid_per_label = [pd.Series(classifications[i * 1000:(i + 1) * 1000]).value_counts().idxmax() for i in range(10)]
max_centroid_per_label

[1, 4, 0, 1, 8, 4, 1, 1, 1, 9]

In [19]:
# Get most common label for each point classified to a centroid (same centroid)
centroid_labels = [np.where(classifications == i)[0] for i in range(10)]
max_label_per_centroid = [pd.Series(prompt_ids[centroid_labels[i]]).value_counts().idxmax() for i in range(10)]
max_label_per_centroid

[3, 1, 9, 3, 6, 4, 9, 10, 5, 10]

In [20]:
max_centroids = [centroid for centroid in max_centroid_per_label for _ in range(1000)]

max_labels = [label for label in max_label_per_centroid for _ in range(1000)]

In [21]:
np.array(max_centroids)

array([1, 1, 1, ..., 9, 9, 9])

In [22]:
np.array(max_labels)

array([ 3,  3,  3, ..., 10, 10, 10])

In [23]:
label_to_centroid = {idx + 1 : max_centroid_per_label[idx] for idx in range(len(max_centroid_per_label))}

centroid_to_label = {idx : max_label_per_centroid[idx] for idx in range(len(max_label_per_centroid))}

In [24]:
label_to_centroid

{1: 1, 2: 4, 3: 0, 4: 1, 5: 8, 6: 4, 7: 1, 8: 1, 9: 1, 10: 9}

In [25]:
centroid_to_label

{0: 3, 1: 1, 2: 9, 3: 3, 4: 6, 5: 4, 6: 9, 7: 10, 8: 5, 9: 10}

In [26]:
vectorized_map = np.vectorize(centroid_to_label.get)
classifications_to_label = vectorized_map(classifications)

classifications_to_label

array([ 9,  9,  9, ..., 10, 10, 10])

In [27]:
for i in range(10):
    print(f"Prompt {i + 1} Accuracy: ", np.mean(classifications_to_label[i * 1000:(i + 1) * 1000] == (i + 1)))

Prompt 1 Accuracy:  0.367
Prompt 2 Accuracy:  0.0
Prompt 3 Accuracy:  0.988
Prompt 4 Accuracy:  0.178
Prompt 5 Accuracy:  0.425
Prompt 6 Accuracy:  0.356
Prompt 7 Accuracy:  0.0
Prompt 8 Accuracy:  0.0
Prompt 9 Accuracy:  0.368
Prompt 10 Accuracy:  0.956


In [28]:
print(f"Overall Accuracy: ", np.mean(classifications_to_label == prompt_ids))

Overall Accuracy:  0.3638


In [29]:
for i in range(10):
    print(f"Prompt {i + 1} Accuracy: ", np.mean(classifications[i * 1000:(i + 1) * 1000] == max_centroids[i * 1000:(i + 1)*1000]))

Prompt 1 Accuracy:  0.367
Prompt 2 Accuracy:  0.336
Prompt 3 Accuracy:  0.554
Prompt 4 Accuracy:  0.355
Prompt 5 Accuracy:  0.425
Prompt 6 Accuracy:  0.356
Prompt 7 Accuracy:  0.338
Prompt 8 Accuracy:  0.299
Prompt 9 Accuracy:  0.328
Prompt 10 Accuracy:  0.546


In [30]:
print(f"Overall Accuracy: ", np.mean(classifications == max_centroids))

Overall Accuracy:  0.3904


## Measure Conversion Accuracy

In [31]:
label_to_centroid_vectorized_map = np.vectorize(label_to_centroid.get)
centroid_to_label_vectorized_map = np.vectorize(centroid_to_label.get)
prompt_ids_to_centroids = label_to_centroid_vectorized_map(prompt_ids)
max_centroids_to_labels = centroid_to_label_vectorized_map(max_centroids)

In [32]:
np.mean(max_centroids == prompt_ids_to_centroids) #100% - max_centroids is the prompt ids converted to the max centroid corresponding to each label

1.0

In [33]:
np.mean(max_centroids_to_labels == prompt_ids) #<100%

0.5